In [1]:
require 'torch'
require 'nn'
require 'rnn'
require 'csvigo'
require 'cutorch'
require 'cunn'
require 'cunnx'

In [2]:
model = 'lstm'
nepochs = 200
K_tokens = 20
J_sentences = 10
batch_size = 100
thresh = 0.00
embed_dim = 100
learning_rate = 0.1
print_every = 1
epsilon = 1
usecuda = false
epsilon = 1
cuts = 4
base_explore_rate = 0.0
skip_rate = 0.
metric = 'f1'

In [3]:
--- Loading utility script
dofile("utils.lua")
dofile("model_utils.lua")

torch.manualSeed(420)
math.randomseed(420)

data_path = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/'

query_fn = data_path .. 'queries_numtext.csv'
query_file =  csvigo.load({path = query_fn, mode = "large", verbose = false})
queries = buildTermDocumentTable(query_file, nil)

pakistan = {
        ['inputs'] = '2012_pakistan_garment_factory_fires_first_sentence_numtext2.csv',
        ['nuggets'] ='pakistan_nuggets_numtext.csv',
        ['query'] = queries[2],
        ['query_name'] = 'pakistan'
}
aurora = {
        ['inputs'] = '2012_aurora_shooting_first_sentence_numtext2.csv', 
        ['nuggets'] = 'aurora_nuggets_numtext.csv',
        ['query'] = queries[3],
        ['query_name'] = 'aurora'
}
sandy = {
        ['inputs'] = 'hurricane_sandy_first_sentence_numtext2.csv',
        ['nuggets'] ='sandy_nuggets_numtext.csv',
        ['query'] = queries[7],
        ['query_name'] = 'sandy'
}

inputs = {
        aurora, 
        -- pakistan,
        -- sandy
    }
--- Only using epsilon greedy strategy for (nepochs/cuts)% of the epochs
delta = 1./(nepochs/cuts) 
crit = nn.MSECriterion()

...Utils file loaded	


In [4]:
input_path, query_file, batch_size, nepochs, inputs = data_path, query_file, batch_size, nepochs, inputs
model, crit, thresh, embed_dim, epsilon, delta = model, crit, thresh, embed_dim, epsilon, delta
base_explore_rate, print_every, learning_rate, J_sentences, K_tokens, use_cuda =base_explore_rate, print_every,learning_rate, J_sentences, K_tokens, usecuda
skiprate, emetric =  skip_rate, metric

In [5]:
if use_cuda then
  Tensor = torch.CudaTensor
  LongTensor = torch.CudaLongTensor
  crit = crit:cuda()
  print("...running on GPU")
else
  Tensor = torch.Tensor
  LongTensor = torch.LongTensor
  print("...running on CPU")
end

...running on CPU	


In [76]:
vocab_size = 0
maxseqlen = 0
maxseqlenq = getMaxseq(query_file)

action_query_list = {}
yrougue_query_list = {}
pred_query_list = {}

for query_id = 1, #inputs do
    input_fn = inputs[query_id]['inputs']
    nugget_fn = inputs[query_id]['nuggets']

    input_file = csvigo.load({path = input_path .. input_fn, mode = "large", verbose = false})
    nugget_file = csvigo.load({path = input_path .. nugget_fn, mode = "large", verbose = false})

    vocab_sized = getVocabSize(input_file)
    vocab_sizeq = getVocabSize(query_file)
    vocab_size = math.max(vocab_size, vocab_sized, vocab_sizeq)

    maxseqlend = getMaxseq(input_file)
    maxseqlen = math.max(maxseqlen, maxseqlenq, maxseqlend)
    action_list = torch.totable(torch.round(torch.rand(#input_file)))
    action_list[1] = 0

    --- initialize the query specific lists
    action_query_list[query_id] = action_list
    yrougue_query_list[query_id] = torch.totable(torch.randn(#input_file, 2)) --- Actual
    pred_query_list[query_id] = torch.totable(torch.zeros(#input_file, 2))    --- Predicted
end

In [47]:
-- model  = build_model(model, vocab_size, embed_dim, 2, use_cuda)
model  = build_model2(model, vocab_size, embed_dim, 1, use_cuda)

In [77]:
epoch = 0
minibatch = 1
query_id = 1

In [49]:
unpackZeros(torch.totable(summary:t()[1])),
unpackZeros(torch.totable(sentences:t()[1])),
actions[1]

{
  1 : 2
  2 : 1
  3 : 6
  4 : 25
  5 : 8
  6 : 24
  7 : 15
  8 : 17
  9 : 20
  10 : 24
  11 : 10
  12 : 7
  13 : 9
  14 : 12
}
{
  1 : 2
  2 : 1
  3 : 6
  4 : 25
  5 : 8
  6 : 24
  7 : 15
  8 : 17
  9 : 20
  10 : 24
  11 : 10
  12 : 7
  13 : 9
  14 : 12
}
 1
[torch.DoubleTensor of size 1]



In [123]:
tst2 = buildCurrentSummary(torch.totable(actions1), xout, K_tokens * J_sentences)

In [125]:
tst2[1]

{
}


In [150]:
function buildCurrentSummary(preds, xs, K)
    local out = {}
    for i = 1, #xs do
        if i == 1 then
            out[i] = {}
        elseif i == 2 then
            tmp = x_or_pass(preds[i-1], unpackZeros(xs[i-1])) 
            out[i] = getLastKElements(tmp, K)
        else 
            local tmp = tableConcat(out[i-2], x_or_pass(preds[i-1], unpackZeros(xs[i-1])))
            out[i] = getLastKElements(tmp, K)
        end 
    end
    return out 
end

function buildPredSummary2(preds, xs, K)
    --- This function is used to map the token indices to extract the summary
    --- and produceds {token_id, 0, token_id} from any given *selected* sentence
    local out = {}
    for i=1, #xs do
        if i == 1 then 
            out[i] = x_or_pass(preds[i][1], unpackZeros(xs[i]))
        else 
            --- Update it by adding xs_i and out_{i-1}
            local tmp = tableConcat(out[i-1], x_or_pass(preds[i][1], unpackZeros(xs[i])))
            --- Getting the last K tokens because we want to keep last K tokens
            out[i] =  getLastKElements(tmp, K)
        end
    end
    return out
end

In [146]:
tst1 = buildPredSummary(torch.totable(actions1), xout, K_tokens * J_sentences)

In [133]:
atmp = geti_n(torch.totable(actions1),4, 6 )
xtmp = geti_n(xout, 4, 6)

In [134]:
atmp, xtmp

{
  1 : 
    {
      1 : 1
    }
  2 : 
    {
      1 : 0
    }
  3 : 
    {
      1 : 1
    }
}
{
  1 : 
    {
      1 : 4801
      2 : 17
      3 : 143
      4 : 124
      5 : 79
      6 : 837
      7 : 1452
      8 : 9
      9 : 24
      10 : 426
      11 : 266
      12 : 124
      13 : 5872
      14 : 9
      15 : 432
      16 : 79
      17 : 24
      18 : 1052
      19 : 8742
      20 : 9
    }
  2 : 
    {
      1 : 193
      2 : 192
      3 : 11054
      4 : 40
      5 : 4095
      6 : 20
      7 : 932
      8 : 1
      9 : 3252
      10 : 29
      11 : 47
      12 : 138
      13 : 131
      14 : 9
      15 : 1038
      16 : 24
    }
  3 : 
    {
      1 : 78
      2 : 174
      3 : 7192
      4 : 24
      5 : 4801
      6 : 1014
      7 : 99
      8 : 2872
      9 : 20
      10 : 24
      11 : 5876
      12 : 1052
      13 : 7283
      14 : 24
    }
}


In [149]:
atmp[1]

{
  1 : 1
}


In [148]:
x_or_pass(atmp[1], xtmp[1])

{
}


In [116]:
actions1[1], buildPredSummary(geti_n(torch.totable(actions1),1, 3 ), 
    geti_n(xout, 1, 3), K_tokens * J_sentences)

 1
[torch.DoubleTensor of size 1]

{
  1 : table: 0x782e39b0
  2 : table: 0x60e5f750
  3 : table: 0x22b8e210
}


In [117]:
actions1[1], buildPredSummary(torch.totable(actions1)[1], {xout[1]}, K_tokens * J_sentences)

 1
[torch.DoubleTensor of size 1]

{
  1 : 
    {
      1 : 9191
      2 : 156
      3 : 116
      4 : 1900
      5 : 945
      6 : 27
      7 : 54
      8 : 392
      9 : 5368
      10 : 1
      11 : 420
      12 : 1014
      13 : 3655
      14 : 945
    }
}


In [98]:
actions0[1], buildPredSummary(torch.totable(actions0)[1], {xout[1]}, K_tokens * J_sentences)

 0
[torch.DoubleTensor of size 1]

{
  1 : table: 0x5fbbdce8
}


In [84]:
summary1[1], actions1[1]

 0
[torch.DoubleTensor of size 1]

 1
[torch.DoubleTensor of size 1]



In [82]:
loss = 0.                    --- Compute a new MSE loss each time
--- Grabbing all of the input data
qs = inputs[query_id]['query']
input_file = csvigo.load({path = input_path .. inputs[query_id]['inputs'], mode = "large", verbose = false})
nugget_file = csvigo.load({path = input_path .. inputs[query_id]['nuggets'], mode = "large", verbose = false})
nuggets = buildTermDocumentTable(nugget_file, nil)

--- Building table of all of the input sentences
xtdm  = buildTermDocumentTable(input_file, K_tokens)

--- Extracting the query specific summaries, actions, and rougue
action_list = action_query_list[query_id]
yrouge = yrougue_query_list[query_id] 
preds = pred_query_list[query_id] 

nbatches = torch.floor( #input_file / batch_size)

--- Initializing rouge metrics at time {t-1} and save scores, reset each new epoch
r_t1 , p_t1, f_t1 = 0., 0., 0.
for minibatch = 1, nbatches do
    if minibatch == 1 then
        -- Need to skip the first row because it says "Text"
        nstart = 2
        nend = torch.round(batch_size * minibatch)
    end
    if minibatch > 1 and minibatch < nbatches then 
        nstart = nend + 1
        nend = torch.round(batch_size * minibatch)
    end
    if minibatch == nbatches then 
        nstart = nend + 1
        nend = #input_file
    end
    --- Processing the input data to get {query, input_sentences, summary, actions}
    action_out = geti_n(action_list, nstart, nend)                
    xout  = geti_n(xtdm, nstart, nend)    --- Extracting the mini-batch from our input sentences
    xs  = padZeros(xout, K_tokens)    --- Padding the data by K tokens because we chose this as the max value
    qs2 = padZeros({qs}, 5)
    qrep = repeatTable(qs2[1], #xs)
--     sumry_list = padZeros(buildCurrentSummary(action_out, xout, K_tokens * J_sentences), K_tokens * J_sentences)
    sumry_list = padZeros(buildPredSummary(action_out, xout, K_tokens * J_sentences), K_tokens * J_sentences)
    --- Inserting data into tensors
    summary0 = LongTensor(sumry_list):t()
    sentences = LongTensor(xs):t()
    query = LongTensor(qrep):t()
    actions0 = Tensor(action_out):resize(#xs, 1)
    
    print(unpackZeros(torch.totable(summary0:t()[1])))
    --- Forward pass to estimate expected rougue)
    pred_rougue0 = model:forward({sentences, summary0, query, actions0})
    
    actions1 = torch.abs(actions0 - 1):resize(#xs, 1)
    --- need to update this to rerun it
    action_list = updateTable(action_list, torch.totable(actions1), nstart)
    
    sumry_list = padZeros(buildPredSummary(action_out, xout, J_sentences * K_tokens), J_sentences * K_tokens)
    summary1 = LongTensor(sumry_list):t()

    print(unpackZeros(torch.totable(summary1:t()[1])))
    --- Score under oposite action
    pred_rougue1 = model:forward({sentences, summary1, query, actions1})

    pred_rougue = Tensor(pred_rougue0):cat(Tensor(pred_rougue1), 2)
    pred_rougue = torch.totable(pred_rougue)

    pred_rougue, labels, opt_action = score_model2(pred_rougue, xout, epsilon, thresh, skiprate)
    
    --- Note setting the skip_rate = 0 means no random skipping of delta calculation
--     labels, opt_action = score_model(torch.totable(pred_rougue), xout, epsilon, thresh, skiprate, emetric)

    -- Updating our bookkeeping tables
    yrouge = updateTable(yrouge, torch.totable(labels), nstart)
    preds =  updateTable(preds, pred_rougue, nstart)
    action_list = updateTable(action_list, opt_action, nstart)
end

{
  1 : 2
  2 : 1
  3 : 6
  4 : 25
  5 : 8
  6 : 24
  7 : 15
  8 : 17
  9 : 20
  10 : 24
  11 : 10
  12 : 7
  13 : 9
  14 : 12
}


{
  1 : 2
  2 : 1
  3 : 6
  4 : 25
  5 : 8
  6 : 24
  7 : 15
  8 : 17
  9 : 20
  10 : 24
  11 : 10
  12 : 7
  13 : 9
  14 : 12
}


executing deterministic policy	


{
}


{
}


...

{
}


executing deterministic policy	


{
}


{
}


executing deterministic policy	


In [1]:
pred_rougue1[1], pred_rougue0[1]

[string "local f = function() return pred_rougue1[1], ..."]:1: attempt to index global 'pred_rougue1' (a nil value)
stack traceback:
	[string "local f = function() return pred_rougue1[1], ..."]:1: in function 'f'
	[string "local f = function() return pred_rougue1[1], ..."]:1: in main chunk
	[C]: in function 'xpcall'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:210: in function <...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:174>
	...ojavierarceo/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:389: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x0103081b90: 

In [ ]:
pred_rougue = Tensor(pred_rougue0):cat(Tensor(pred_rougue1), 2)
sumTable(opt_action)

In [39]:
--- Rerunning on the scoring on the full data and rescoring cumulatively
--- Execute policy and evaluation based on our E[ROUGUE] after all of the minibatches
    --- Notice that pred_rougue gives us our optimal action by returning
    ---  E[ROUGUE | Select ] > E[ROUGUE | Skip]
predsummary = buildPredSummary(action_list, xtdm, nil)
predsummary = predsummary[#predsummary]

rscore = rougeRecall({predsummary}, nuggets)
pscore = rougePrecision({predsummary}, nuggets)
fscore = rougeF1({predsummary}, nuggets)

In [40]:
rscore, pscore, fscore

0	0	0	


In [ ]:
--- Updating variables
action_query_list[query_id] = action_list
yrougue_query_list[query_id] = yrouge
pred_query_list[query_id] = preds

if (epoch % print_every)==0 then
    perf_string = string.format(
        "Epoch %i, epsilon = %.3f, sum(y)/len(y) = %i/%i, {Recall = %.6f, Precision = %.6f, F1 = %.6f}, query = %s", 
        epoch, epsilon, sumTable(action_list), #action_list, rscore, pscore, fscore, inputs[query_id]['query_name']
        )
    print(perf_string)
end

--- creating the indices we want
xindices = {}
for i=1, 100 do
    xindices[i] = math.random(2, #xtdm)
end
--- Have to skip over stupid header
xout = getIndices(xtdm, xindices)
action_out = getIndices(action_list, xindices)
labels = Tensor(getIndices(yrouge, xindices)):resize(#xout, 1)
pred_rougue = Tensor(getIndices(preds, xindices)):resize(#xout, 1)

xs  = padZeros(xout, K_tokens)    --- Padding the data by K tokens because we chose this as the max value
qs2 = padZeros({qs}, 5)
qrep = repeatTable(qs2[1], #xs)

sumry_list = buildCurrentSummary(action_out, xs, K_tokens * J_sentences)
-- local sumry_list = buildPredSummary(action_out, xs, K_tokens * J_sentences)
summary = LongTensor(padZeros(sumry_list, K_tokens * J_sentences)):t()

--- Inserting data into tensors            
sentences = LongTensor(xs):t()
query = LongTensor(qrep):t()

--- Backprop model
loss = loss + crit:forward(pred_rougue, labels)
grads = crit:backward(pred_rougue, labels)
model:zeroGradParameters()
model:backward({sentences, summary, query}, grads)
model:updateParameters(learning_rate)


In [ ]:
summary:t()[1]:sum(), unpackZeros(torch.totable(sentences:t()[1])), query:t()[1]

In [ ]:
torch.totable(pred_rougue)[1]